In [2]:
!pip install SQLAlchemy

In [3]:
!pip install mysql-connector-python

In [10]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

from getpass import getpass

In [12]:
password = getpass.getpass()

 ········


In [14]:
#1.Establish a connection between Python and the Sakila database.

cnx=mysql.connector.connect(user="root",
                           password=password,
                           host="localhost",
                           database="sakila")

In [16]:
cnx.is_connected()

True

In [18]:
cursor=cnx.cursor()

In [ ]:
"""
2.
Write a Python function called rentals_month that retrieves rental data for a given month and year (passed as parameters) 
from the Sakila database as a Pandas DataFrame. The function should take in three parameters:

engine: an object representing the database connection engine to be used to establish a connection to the Sakila database.
month: an integer representing the month for which rental data is to be retrieved.
year: an integer representing the year for which rental data is to be retrieved.
The function should execute a SQL query to retrieve the rental data for the specified month and year from the 
rental table in the Sakila database, and return it as a pandas DataFrame.

"""

In [98]:
def rentals_month(cnx, month, year):
    # Query to get the info
    query = f"""
    SELECT rental_id, rental_date, inventory_id, customer_id, return_date, staff_id
    FROM sakila.rental
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year};
    """
    
    # read sql into a df
    df_rentals = pd.read_sql(query, cnx)
    
    return df_rentals

In [100]:
df_may_rentals = rentals_month(cnx, 5, 2005)  
df_june_rentals = rentals_month(cnx, 6, 2005)

C:\Users\Tanguy\AppData\Local\Temp\ipykernel_1812\1316048574.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_rentals = pd.read_sql(query, cnx)


In [ ]:
"""
Develop a Python function called rental_count_month that takes the DataFrame provided by rentals_month as 
input along with the month and year and returns a new DataFrame containing the number of rentals made by 
each customer_id during the selected month and year.

The function should also include the month and year as parameters and use them to name the new column 
according to the month and year, for example, if the input month is 05 and the year is 2005, the column 
name should be "rentals_05_2005".

Hint: Consider making use of pandas groupby()
"""

In [102]:
def rental_count_month(df_rentals, month, year):
    # Group by customer_id and count rentals
    rental_counts = df_rentals.groupby('customer_id').size().reset_index(name='rental_count')
    
    # Create a new column name based on month and year
    column_name = f'rentals_{month:02d}_{year}'
    
    # Rename the rental_count column to the new column name
    rental_counts = rental_counts.rename(columns={'rental_count': column_name})
    
    return rental_counts

In [104]:
df_rentals_may = rental_count_month(df_may_rentals, 5, 2005) 
df_rentals_june = rental_count_month(df_june_rentals, 6, 2005) 

In [ ]:
"""
4.Create a Python function called compare_rentals that takes two DataFrames as input containing the number of rentals 
made by each customer in different months and years. The function should return a combined DataFrame with 
a new 'difference' column, which is the difference between the number of rentals in the two months.
"""

In [106]:
def compare_rentals(df_rentals_month1, df_rentals_month2):
    # Merge the two DataFrames on customer_id
    combined_df = pd.merge(
        df_rentals_month1,
        df_rentals_month2,
        on='customer_id',
        how='outer',  # Outer to include all customers
    )
    
    # Calculate the difference in rentals
    combined_df['difference'] = combined_df[f'rentals_05_2005'].fillna(0) - combined_df[f'rentals_06_2005'].fillna(0)
    
    # Return the combined DataFrame
    return combined_df[['customer_id', f'rentals_05_2005', f'rentals_06_2005', 'difference']]


In [110]:
comparison_df = compare_rentals(df_rentals_may, df_rentals_june)  
pd.DataFrame(comparison_df)

,customer_id,rentals_05_2005,rentals_06_2005,difference
0,1,2.0,7.0,-5.0
1,2,1.0,1.0,0.0
2,3,2.0,4.0,-2.0
3,4,NaN,6.0,-6.0
4,5,3.0,5.0,-2.0
...,...,...,...,...
593,595,1.0,2.0,-1.0
594,596,6.0,2.0,4.0
595,597,2.0,3.0,-1.0
596,598,NaN,1.0,-1.0
